<a href="https://colab.research.google.com/github/gogamid/ml-notebooks/blob/main/continualai/8-putting-all-together.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install avalanche-lib

In [2]:
#!wandb login --relogin

In [3]:
import wandb
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import accuracy_metrics, loss_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, WandBLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.supervised import Naive

scenario = SplitMNIST(n_experiences=5, fixed_class_order=[0,1,2,3,4,5,6,7,8,9])
model = SimpleMLP(num_classes=scenario.n_classes)

interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    loggers=[interactive_logger]
)

In [4]:
from avalanche.training.plugins import LwFPlugin, EWCPlugin, SynapticIntelligencePlugin, CWRStarPlugin
wandb_logger = WandBLogger(project_name="reg-strategies", run_name="cwr")
eval_plugin.loggers.append(wandb_logger)

# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=500, train_epochs=1, eval_mb_size=100,
    evaluator=eval_plugin,
    plugins=[CWRStarPlugin(model)]
)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(scenario.test_stream))
wandb.finish()

wandb: Currently logged in as: gogamid. Use `wandb login --relogin` to force relogin


Starting experiment...
Start of experience:  0
Current Classes:  [0, 1]
-- >> Start of training phase << --
100%|██████████| 26/26 [00:07<00:00,  3.52it/s]
Epoch 0 ended.
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 22/22 [00:01<00:00, 19.61it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9962
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 21/21 [00:01<00:00, 20.95it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.0000
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 19/19 [00:00<00:00, 20.30it/s]
> Eval on experience 2 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002 = 0.0000
-- Starting eval on ex

Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000,█▅▅▄▁
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001,▁███▇
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002,▁▁██▇
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp003,▁▁▁██
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp004,▁▁▁▁█
Top1_Acc_Stream/eval_phase/test_stream/Task000,▁▃▄▇█
TrainingExperience,▁▃▅▆█
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000,0.97447
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001,0.69882
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002,0.41622
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp003,0.79406
